<a href="https://colab.research.google.com/github/MathMachado/eDreams/blob/master/eDreams.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install & Load Main Python libraries



In [0]:
!pip install bamboolib

In [0]:
import pandas as pd
import numpy as np

import matplotlib
import bamboolib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Load dataframes: training & test sample

In [0]:
url_train= "https://raw.githubusercontent.com/MathMachado/eDreams/master/Dataframes/train.csv?token=AGDJQ67D2WENLE2YEMFJBPC5SHK26"
url_test= "https://raw.githubusercontent.com/MathMachado/eDreams/master/Dataframes/test.csv?token=AGDJQ6YYHJOGUATPKWUASIK5SSGF6"

# Stacking training and validation samples for a single treatment
df_train= pd.read_csv(url_train, sep= ";", index_col='ID', parse_dates = ['DEPARTURE', 'ARRIVAL'])
df_test= pd.read_csv(url_test, sep= ";", index_col='ID', parse_dates = ['DEPARTURE', 'ARRIVAL'])

# Resetting the test sample indices
df_test.index= range(50000, 80000)

# merge train and test
df = df_train.append(df_test, sort= True)

# Records training and test dataframe indexes to separate these dataframes later
train_index = df_train.index
test_index = df_test.index

In [0]:
df.shape

In [0]:
df.head()

In [0]:
df.tail()

In [0]:
df_test.head()

In [0]:
df_test.tail()

In [0]:
df.info()

# Data Preparation

## Treating date variables
> Since there is no information regarding the year of the transaction, I will assume that the transactions are from 2018 or 2019. I will assign the year conveniently from the analysis of the variables DEPARTURE and ARRIVAL.

In [0]:
df2= df.copy()
df2['DEPARTURE_WITH_YEAR']= df2['DEPARTURE'] +'/2018'
df2['ARRIVAL_WITH_YEAR']= df2['ARRIVAL'] +'/2018'
df2['ARRIVAL_WITH_YEAR_FIXED']= df2['ARRIVAL'] +'/2019'

df2['DEPARTURE_WITH_YEAR']= pd.to_datetime(df2['DEPARTURE_WITH_YEAR'])
df2['ARRIVAL_WITH_YEAR']= pd.to_datetime(df2['ARRIVAL_WITH_YEAR'])
df2['ARRIVAL_WITH_YEAR_FIXED']= pd.to_datetime(df2['ARRIVAL_WITH_YEAR_FIXED'])
df2.head()

As we do not have year information, in some cases/rows we have ARRIVAL < DEPARTURE. Let's take a look in some cases where ARRIVAL < DEPARTURE:

In [0]:
df3= df2.copy()

# I created the variable IS_ARRIVAL_BEFORE_DEPARTURE to help us identify when ARRIVAL < DEPARTURE:
df3['IS_ARRIVAL_BEFORE_DEPARTURE']= df3['ARRIVAL_WITH_YEAR']<df3['DEPARTURE_WITH_YEAR']

# Fixing cases when ARRIVAL < DEPARTURE
df3.loc[df3['IS_ARRIVAL_BEFORE_DEPARTURE']== True, 'ARRIVAL_WITH_YEAR']= df3['ARRIVAL_WITH_YEAR_FIXED']
df3[['ARRIVAL', 'DEPARTURE', 'DEPARTURE_WITH_YEAR', 'ARRIVAL_WITH_YEAR']][df3['IS_ARRIVAL_BEFORE_DEPARTURE']== True].head()

> Take for example line 15 (output above):
* DEPARTURE= December 15th;
* ARRIVAL= January 29th.

> Without information for the year, then ARRIVAL < DEPARTURE. However, look at the variables DEPARTURE_WITH_YEAR and ARRIVAL_WITH_YEAR above:
* DEPARTURE_WITH_YEAR= December 15th of 2018;
* ARRIVAL_WITH_YEAR= January 29th of 2019.

In this case, we fixed the problem.



In [0]:
# Drop the unnecessary variables:
df3= df3.drop(columns= ['DEPARTURE', 'ARRIVAL', 'ARRIVAL_WITH_YEAR_FIXED'])

Next, we calculate the variable ARRIVAL_DEPARTURE = ARRIVAL_WITH_YEAR - DEPARTURE_WITH_YEAR:

In [0]:
# Calculate the variable ARRIVAL_DEPARTURE:
df3['ARRIVAL_DEPARTURE']= (df3['ARRIVAL_WITH_YEAR']-df3['DEPARTURE_WITH_YEAR']).dt.days.astype(int)

# Show some cases:
df3[['DEPARTURE_WITH_YEAR', 'ARRIVAL_WITH_YEAR', 'ARRIVAL_DEPARTURE']].head() #[df3['IS_ARRIVAL_BEFORE_DEPARTURE']== True].head()

> Something strange with the variable ARRIVAL_DEPARTURE. Take a look at the line 2 (above). We have:
* DEPARTURE_WITH_YEAR= 2018-07-29
* ARRIVAL_WITH_YEAR= 2018-08-19

It is 21 days between DEPARTURE and ARRIVAL!

Let's take a look in some statistics below. For example, let's look at the proportion of cases where ARRIVAL_DEPARTURE > 5. I am using 5 as an example, but I consider 5 a long time between departure and arrival.

In [0]:
df3_Zoom= df3[df3['ARRIVAL_DEPARTURE'] > 5]
df3_Zoom.shape[0]

Strangely, many cases over 5 days. 

Below, I present the distribution of the ARRIVAL_DEPARTURE variable. As we can see, there are cases older than 50 days!

In [0]:
sns.distplot(df3['ARRIVAL_DEPARTURE'])

Below I present some descriptive statistics for ARRIVAL_DEPARTURE. The median of the variable by type of HAUL_TYPE makes more sense. However, I still find median= 10 a long time in the case of HAUL_TYPE= "INTERCONTINENTAL".

In [0]:
df3.groupby('HAUL_TYPE').agg({'ARRIVAL_DEPARTURE': ['min', 'median', 'mean', 'max', 'count']})

Let's take a look at the Boxplot:

In [0]:
plt.rcdefaults()
sns.catplot(y='ARRIVAL_DEPARTURE', kind="box", data=df3, height=4, aspect=1.5)
plt.show()

Below, the boxplot of ARRIVAL_DEPARTURE by HAUL_TYPE:

In [0]:
# make boxplot with Catplot
plt.rcdefaults()
sns.catplot(x='HAUL_TYPE', y='ARRIVAL_DEPARTURE', kind="box", data=df3, height=4, aspect=1.5)
plt.show()

I will do the following, I will calculate the median disregarding the outliers of the variable ARRIVAL_DEPARTURE by HAUL_TYPE.

In [0]:
df4= df3.copy()
df_DOMESTIC= df4[['HAUL_TYPE', 'ARRIVAL_DEPARTURE']][df4['HAUL_TYPE']== 'DOMESTIC']
df_CONTINENTAL= df4[['HAUL_TYPE', 'ARRIVAL_DEPARTURE']][df4['HAUL_TYPE']== 'CONTINENTAL']
df_INTERCONTINENTAL= df4[['HAUL_TYPE', 'ARRIVAL_DEPARTURE']][df4['HAUL_TYPE']== 'INTERCONTINENTAL']

In [0]:
# Let's take a look at one of it: df_DOMESTIC:
df_DOMESTIC.head()

Function to detect Outliers based on IQR-Score:

In [0]:
# Function that identify outlier using IQR-Score:
def IQR_Score_Outlier_Detect(column):
    global df_Temp
    
    Q1 = df_Temp[column].quantile(0.25)
    Q3 = df_Temp[column].quantile(0.75)
    IQR = Q3 - Q1

    df_Temp = df_Temp[~((df_Temp[column] < (Q1-1.5*IQR)) |(df_Temp[column] > (Q3+1.5*IQR)))]   

In [0]:
df_Temp= df_DOMESTIC
IQR_Score_Outlier_Detect('ARRIVAL_DEPARTURE')
df_Temp.head()

In [0]:
# make boxplot with Catplot
plt.rcdefaults()
sns.catplot(y='ARRIVAL_DEPARTURE', kind="box", data=df_Temp, height=4, aspect= 1.5)
plt.show()

As you can see above, we removed the outliers from the dataframe when HAUL_TYPE= "DOMESTIC".

In [0]:
df_Temp.groupby('HAUL_TYPE').agg({'ARRIVAL_DEPARTURE': ['min', 'median', 'mean', 'max', 'count']})

In the case of HAUL_TYPE = 'DOMESTIC', we now have median = 1, according to the IQR-Score criteria. Let's see how the other cases look:

In [0]:
df_Temp= df_CONTINENTAL
IQR_Score_Outlier_Detect('ARRIVAL_DEPARTURE')
df_Temp.head()

In [0]:
# make boxplot with Catplot
plt.rcdefaults()
sns.catplot(y='ARRIVAL_DEPARTURE', kind="box", data=df_Temp, height=4, aspect= 1.5)
plt.show()

As you can see above, we removed the outliers from the dataframe when HAUL_TYPE= "CONTINENTAL".

In [0]:
df_Temp.groupby('HAUL_TYPE').agg({'ARRIVAL_DEPARTURE': ['min', 'median', 'mean', 'max', 'count']})

In the case of HAUL_TYPE = "CONTINENTAL", we have median = 3. Before it was 3, but the average has reduced a lot. Finally, let's look at the last class: HAUL_TYPE = "INTERCONTINENTAL":

In [0]:
df_Temp= df_INTERCONTINENTAL
IQR_Score_Outlier_Detect('ARRIVAL_DEPARTURE')
df_Temp.head()

In [0]:
# make boxplot with Catplot
plt.rcdefaults()
sns.catplot(y='ARRIVAL_DEPARTURE', kind="box", data=df_Temp, height=4, aspect= 1.5)
plt.show()

As you can see above, we removed the outliers from the dataframe when HAUL_TYPE= "INTERCONTINENTAL".

In [0]:
df_Temp.groupby('HAUL_TYPE').agg({'ARRIVAL_DEPARTURE': ['min', 'median', 'mean', 'max', 'count']})

Finally, we have median = 9 for HAUL_TYPE = "INTERCONTINENTAL".

> **DECISION**: As I said earlier, I find the behavior of this variable strange. I initially considered deleting this variable, but now I've decided to keep it in the analysis. However, I will do the following transformation:
* HAUL_TYPE = "DOMESTIC": If ARRIVAL_DEPARTURE> Median then ARRIVAL_DEPARTURE = Median;
* HAUL_TYPE = "CONTINENTAL": If ARRIVAL_DEPARTURE> Median then ARRIVAL_DEPARTURE = Median;
* HAUL_TYPE = "INTERCONTINENTAL": If ARRIVAL_DEPARTURE> Median then ARRIVAL_DEPARTURE = Median;

Replacing all values of where ARRIVAL_DEPARTURE > Median:

In [0]:
#df5= df4.copy()
#df5['ARRIVAL_DEPARTURE'].loc[((df5['ARRIVAL_DEPARTURE'] > 1) & (df5['HAUL_TYPE']== 'DOMESTIC'))] = 1
#df5['ARRIVAL_DEPARTURE'].loc[((df5['ARRIVAL_DEPARTURE'] > 3) & (df5['HAUL_TYPE']== 'CONTINENTAL'))] = 3
#df5['ARRIVAL_DEPARTURE'].loc[((df5['ARRIVAL_DEPARTURE'] > 9) & (df5['HAUL_TYPE']== 'INTERCONTINENTAL'))] = 9

In [0]:
df5= df4.copy()
df5['ARRIVAL_DEPARTURE'] = np.where(((df5['ARRIVAL_DEPARTURE'] > 1) & (df5['HAUL_TYPE']== 'DOMESTIC')), 1, df5['ARRIVAL_DEPARTURE'])
df5['ARRIVAL_DEPARTURE'] = np.where(((df5['ARRIVAL_DEPARTURE'] > 3) & (df5['HAUL_TYPE']== 'CONTINENTAL')), 3, df5['ARRIVAL_DEPARTURE'])
df5['ARRIVAL_DEPARTURE'] = np.where(((df5['ARRIVAL_DEPARTURE'] > 9) & (df5['HAUL_TYPE']== 'INTERCONTINENTAL')), 9, df5['ARRIVAL_DEPARTURE'])

# Checking statistics:
df5.groupby('HAUL_TYPE').agg({'ARRIVAL_DEPARTURE': ['min', 'median', 'mean', 'max', 'count']})

In [0]:
df5.head()

In [0]:
# Deleting Unneeded Variables
df5= df5.drop(columns= ['TIMESTAMP','DEPARTURE_WITH_YEAR','ARRIVAL_WITH_YEAR', 'IS_ARRIVAL_BEFORE_DEPARTURE'], axis= 1)

## Handling Missing Values

In [0]:
df5.info()

> Apparently we have some problems from Missing Values to DEVICE. Don't worry about the Missing values of the EXTRA_BAGGAGE variable that is our response variable and the 30,000 Missing values presented come from the test sample and are just the values we want to predict.

In [0]:
# Converting column DISTANCE to numeric. For this purpose, I'll cut the distance in the ","
df6= df5.copy()
df6[['DISTANCE_2','DISTANCE_REST']] = df6['DISTANCE'].str.split(",",expand=True)
df6['DISTANCE_2']= pd.to_numeric(df6['DISTANCE_2'])
df6[['HAUL_TYPE','DISTANCE','DISTANCE_2','DISTANCE_REST']].head(10)

In [0]:
df6.groupby('HAUL_TYPE').agg({'DISTANCE_2': ['min', 'median', 'max', 'count']})

Something strange with the minimum of DISTANCE_2. No sense DOMESTIC = 0. Much less INTERCONTINENTAL = 0. Let's investigate this a little further. However, I will work with DISTANCE_2 (following I will rename DISTANCE_2 TO DISTANCE) and disregard DISTANCE_REST.

In [0]:
df6= df6.drop(columns= ['DISTANCE_REST','DISTANCE'], axis= 1)
df6= df6.rename({'DISTANCE_2': 'DISTANCE'}, axis=1)
df6.head()

In [0]:
# How many cases where DISTANCE = 0?
df6[['DISTANCE']][df6['DISTANCE']==0].count()

There are 288 records where DISTANCE = 0. I consider these records to be Missing Values. I will impute missing values based on HAUL_TYPE.

In [0]:
median_by__HAUL_TYPE= df6.groupby('HAUL_TYPE')['DISTANCE'].median()
median_by__HAUL_TYPE

In [0]:
median_DISTANCE_DOMESTIC= median_by__HAUL_TYPE[1]
median_DISTANCE_CONTINENTAL= median_by__HAUL_TYPE[2]

Median for DISTANCE when HAUL_TYPE = 'DOMESTIC' - I will use this value for missing values of DISTANCE when HAUL_TYPE = 'DOMESTIC'.

In [0]:
median_DISTANCE_DOMESTIC

Median for DISTANCE when HAUL_TYPE = 'CONTINENTAL' - I will use this value for missing values of DISTANCE when HAUL_TYPE = 'CONTINENTAL'.

In [0]:
median_DISTANCE_CONTINENTAL

In [0]:
# Identifying Missing Values in DISTANCE. In this case, zeros.
df6.loc[df6['DISTANCE'] == 0, 'DISTANCE']= np.nan

# Checking Missing Values
df6.isna().sum()

Let's treat Missing Values in DISTANCE and DEVICE below:

In [0]:
# Missing Value imputation for DOMESTIC
df6['DISTANCE'] = np.where(((df6['DISTANCE'].isnull()) & (df6['HAUL_TYPE'] =="DOMESTIC")), median_DISTANCE_DOMESTIC, df6['DISTANCE'])

# Missing Value imputation for INTERCONTINENTAL
df6['DISTANCE'] = np.where(((df6['DISTANCE'].isnull()) & (df6['HAUL_TYPE'] =="INTERCONTINENTAL")), median_DISTANCE_CONTINENTAL, df6['DISTANCE'])

# Show some statistics
df6.groupby('HAUL_TYPE').agg({'DISTANCE': ['min', 'median', 'max', 'count']})

In [0]:
# Checking Missing Values
df6.isna().sum()

In [0]:
# Treating Missing Values of DEVICE
df6['DEVICE'].value_counts() 

In [0]:
# Replacing NaN's of DEVICE with 'NO_DEVICE'
df6["DEVICE"].fillna("NO_DEVICE", inplace= True)

# Checking Missing Values
df6.isna().sum()

As we can see above, missing values have been addressed.

In [0]:
# Checking...
df6['DEVICE'].value_counts() 

In [0]:
df7= df6.copy()
df7.head()

# Handling Outliers in DISTANCE using IQR-Score
> Consider the following output:

In [0]:
# Some statistics before outlier treatment in DISTANCE
df7.groupby('HAUL_TYPE').agg({'DISTANCE': ['min', 'median', 'max', 'count']})

In [0]:
sns.distplot(df7['DISTANCE'])

In [0]:
# make boxplot with Catplot
plt.rcdefaults()
sns.catplot(x='HAUL_TYPE', y='DISTANCE', kind="box", data=df7, height=4, aspect=1.5)
plt.show()

As we can see above, we have some outliers in the DISTANCE variable.

In [0]:
from scipy import stats
threshold = 3

df_DOMESTIC= df7[df7['HAUL_TYPE']== 'DOMESTIC']
df_CONTINENTAL= df7[df7['HAUL_TYPE']== 'CONTINENTAL']
df_INTERCONTINENTAL= df7[df7['HAUL_TYPE']== 'INTERCONTINENTAL']

In [0]:
df_DOMESTIC['HAUL_TYPE'].value_counts() 

In [0]:
df_CONTINENTAL['HAUL_TYPE'].value_counts() 

In [0]:
df_INTERCONTINENTAL['HAUL_TYPE'].value_counts() 

Calculating IQR-Score for each HAULT_TYPE:

In [0]:
# IQR-Score
Q1_DOMESTIC = df_DOMESTIC['DISTANCE'].quantile(0.25)
Q3_DOMESTIC = df_DOMESTIC['DISTANCE'].quantile(0.75)
IQR_DOMESTIC = Q3_DOMESTIC - Q1_DOMESTIC

Q1_CONTINENTAL = df_CONTINENTAL['DISTANCE'].quantile(0.25)
Q3_CONTINENTAL = df_CONTINENTAL['DISTANCE'].quantile(0.75)
IQR_CONTINENTAL = Q3_CONTINENTAL - Q1_CONTINENTAL

Q1_INTERCONTINENTAL = df_INTERCONTINENTAL['DISTANCE'].quantile(0.25)
Q3_INTERCONTINENTAL = df_INTERCONTINENTAL['DISTANCE'].quantile(0.75)
IQR_INTERCONTINENTAL = Q3_INTERCONTINENTAL - Q1_INTERCONTINENTAL

In [0]:
# Excluding Outliers based on IQR-Score:
df_DOMESTIC_IQR = df_DOMESTIC[~((df_DOMESTIC['DISTANCE'] < (Q1_DOMESTIC-1.5*IQR_DOMESTIC)) |(df_DOMESTIC['DISTANCE'] > (Q3_DOMESTIC+1.5*IQR_DOMESTIC)))]
df_DOMESTIC_IQR.shape[0]

In [0]:
df_CONTINENTAL.shape[0]

In [0]:
df_CONTINENTAL_IQR = df_CONTINENTAL[~((df_CONTINENTAL['DISTANCE'] < (Q1_CONTINENTAL-1.5*IQR_CONTINENTAL)) |(df_CONTINENTAL['DISTANCE'] > (Q3_CONTINENTAL+1.5*IQR_CONTINENTAL)))]
df_CONTINENTAL_IQR.shape[0]

In [0]:
df_INTERCONTINENTAL.shape[0]

In [0]:
df_INTERCONTINENTAL_IQR = df_INTERCONTINENTAL[~((df_INTERCONTINENTAL['DISTANCE'] < (Q1_INTERCONTINENTAL-1.5*IQR_INTERCONTINENTAL)) |(df_INTERCONTINENTAL['DISTANCE'] > (Q3_INTERCONTINENTAL+1.5*IQR_INTERCONTINENTAL)))]
df_INTERCONTINENTAL_IQR.shape[0]

In [0]:
df_IQR = df_DOMESTIC_IQR.append(df_CONTINENTAL_IQR, sort= True)
df_IQR = df_IQR.append(df_INTERCONTINENTAL_IQR, sort= True)
df_IQR.shape[0]

In [0]:
80000-76135

Based on IQR-Score, we will lose 3,865 rows/records.

In [0]:
df_IQR.head()

In [0]:
# make boxplot with Catplot
plt.rcdefaults()
sns.catplot(x='HAUL_TYPE', y='DISTANCE', kind="box", data=df_IQR, height=4, aspect=1.5)
plt.show()

Note, however, that we do not have as many outliers as before.

Response-variable distribution after outlier treatment by IQR-Score:

In [0]:
# BEFORE Outlier treatment in DISTANCE variable
df7['EXTRA_BAGGAGE'].value_counts() 

In [0]:
# AFTER Outlier treatment in DISTANCE variable
df_IQR['EXTRA_BAGGAGE'].value_counts() 

Small loss ... So I will continue with the database treated for outliers by IQR-Score.

In [0]:
# DISTANCE BEFORE Outlier's treatment
df7.groupby('HAUL_TYPE').agg({'DISTANCE': ['min', 'median', 'max', 'count']})

In [0]:
# DISTANCE AFTER Outlier's treatment
df_IQR.groupby('HAUL_TYPE').agg({'DISTANCE': ['min', 'median', 'max', 'count']})

Note that the median has changed slightly when comparing before and after outlier treatment. I'll continue anyway, because I don't see it as a problem ...

In [0]:
sns.distplot(df_IQR['DISTANCE'])

# Binning numeric features

In [0]:
df8= df_IQR.copy()
df8['EXTRA_BAGGAGE'].value_counts() 

In [0]:
d_Var_Target= {True: 1, False: 0}
df8['EXTRA_BAGGAGE']= df8['EXTRA_BAGGAGE'].map(d_Var_Target)
df8.head()

In [0]:
df8.info()

## Treating numerical variables

In [0]:
df8['ARRIVAL_DEPARTURE'].value_counts()

In [0]:
# make boxplot with Catplot
plt.rcdefaults()
sns.catplot(x='HAUL_TYPE', y='ARRIVAL_DEPARTURE', kind="box", data=df8, height=4, aspect=1.5)
plt.show()

Binning DISTANCE...

In [0]:
df8['DISTANCE_CAT']= pd.cut(df8['DISTANCE'], 10)
df8= df8.drop(columns= ['DISTANCE'], axis= 1)
df8.head()

In [0]:
df8['DISTANCE_CAT'].value_counts()

## Treating categorical variables

In [0]:
df9= df8.copy()
df9.head()

In [0]:
df10 = pd.get_dummies(df9, columns=['DISTANCE_CAT', 'ADULTS','ARRIVAL_DEPARTURE','CHILDREN','GDS','INFANTS','NO_GDS','DEVICE', 'HAUL_TYPE', 'PRODUCT', 'SMS', 'TRAIN', 'TRIP_TYPE', 'WEBSITE'], drop_first=True)
df10.head()

# Modeling

In [0]:
from sklearn.ensemble import VotingClassifier, RandomForestClassifier,AdaBoostClassifier
from sklearn.metrics import accuracy_score, log_loss
from sklearn.neighbors import KNeighborsClassifier, NearestCentroid
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.linear_model import RidgeClassifier, PassiveAggressiveClassifier, SGDClassifier, LogisticRegressionCV, LogisticRegression
from sklearn.naive_bayes import BernoulliNB, MultinomialNB, GaussianNB
from sklearn.neural_network import MLPClassifier # Multi-Layer Perceptron Classifier
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier, RandomForestClassifier, VotingClassifier, RandomForestClassifier, GradientBoostingClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.feature_selection import SelectKBest, chi2
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import Perceptron
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split

## Train/Test Split

The following is the training sample, which we will balance.

In [0]:
df11= df10.copy()
X_train= df11[df11['EXTRA_BAGGAGE'].notna()]
X_train= X_train.drop(columns= ['EXTRA_BAGGAGE'], axis= 1)

y_train= df11[df11['EXTRA_BAGGAGE'].notna()]
y_train= y_train['EXTRA_BAGGAGE']

print(X_train.shape, y_train.shape)

The following is the validation sample:

In [0]:
X_test= df11[df11['EXTRA_BAGGAGE'].isna()]
X_test= X_test.drop(columns= ['EXTRA_BAGGAGE'], axis= 1)

y_test= df11[df11['EXTRA_BAGGAGE'].isna()]
y_test= y_test['EXTRA_BAGGAGE']

print(X_test.shape, y_test.shape)

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
print(X_train.shape, y_train.shape)

Importance Sampling through 'Random Forest':

In [0]:
from sklearn.ensemble import RandomForestClassifier
rf_clf = RandomForestClassifier(n_estimators = 500, max_depth=12)
rf_clf.fit(X_train, y_train)
rf_y_pred = rf_clf.predict(X_test)

pd.Series(rf_clf.feature_importances_, index = X_train.columns).nlargest(12).plot(kind = 'barh',
                                                                               figsize = (10, 10),
                                                                              title = 'Feature importance from RandomForest').invert_yaxis();

As we can see above, the top 10 most important features are:

01. NO_GDS_1;
02. HAUL_TYPE_INTERCONTINENTAL;
03. GDS_1;
04. ADULTS_1;
05. ARRIVAL_DEPARTURE_3;
06. ADULTS_2;
07. TRIP_TYPE_ROUND_TRIP;
08. ARRIVAL_DEPARTURE_9;
09. TRIP_TYPE_ONE_WAY;
10. ARRIVAL_DEPARTURE_2.


Next, we will apply the following estimators / classifiers to the training sample:

In [0]:
l_Estimators = [('KNeighborsClassifier', KNeighborsClassifier(3)),
               ('SVC', SVC(kernel="rbf", C= 10, probability=True)),
               ('NuSVC', NuSVC(probability=True)),
               ('DecisionTreeClassifier', DecisionTreeClassifier()),
               ('RandomForestClassifier', RandomForestClassifier()),
               ('GradientBoostingClassifier', GradientBoostingClassifier()),
               ('RidgeClassifier', RidgeClassifier()),
               ('AdaBoostClassifier', AdaBoostClassifier()),
               ('GaussianNB', GaussianNB()),
               ('BernoulliNB', BernoulliNB()),
               ('PassiveAggressiveClassifier', PassiveAggressiveClassifier()),
               ('LinearSVC', LinearSVC()),
               ('SGDClassifier', SGDClassifier(loss='log', penalty='elasticnet')),
               ('LogisticRegression', LogisticRegression()),
               ('NearestCentroid', NearestCentroid()),
               ('Perceptron', Perceptron()),
               ('MLPClassifier', MLPClassifier(hidden_layer_sizes=(10, 10, 10), max_iter=1000)),
               ('LGBMClassifier', LGBMClassifier()),
               ('BaggingClassifier', BaggingClassifier(KNeighborsClassifier(), max_samples=0.5, max_features=0.5)),
               ('GaussianProcessClassifier', GaussianProcessClassifier()),
               ('XGBClassifier', XGBClassifier(n_estimators= 1000,max_depth= 4,objective='reg:logistic',scale_pos_weight=1)),
               ('ExtraTreesClassifier', ExtraTreesClassifier(n_estimators = 750, max_features = 'sqrt', max_depth = 35,  criterion = 'entropy', random_state = 20111974))]

In [0]:
from sklearn.model_selection import cross_val_score, StratifiedKFold, GridSearchCV
from sklearn.metrics import f1_score

In [0]:
from sklearn.tree import DecisionTreeClassifier

DT= DecisionTreeClassifier(max_depth= 5)
DT.fit(X_train,y_train)
y_pred = DT.predict(X_test)
DT.score(X_test,y_test)

In [0]:
f1_score(y_test, y_pred, average= 'weighted')

In [0]:
from sklearn.ensemble import RandomForestClassifier

RF= RandomForestClassifier(max_depth= 5, n_estimators= 1000, n_jobs= -1)
RF.fit(X_train,y_train)
y_pred = RF.predict(X_test)
RF.score(X_test,y_test)

In [0]:
f1_score(y_test, y_pred, average= 'weighted')

In [0]:
from sklearn.ensemble import AdaBoostClassifier

AB = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=8),n_estimators=600)
AB.fit(X_train,y_train)
y_pred = AB.predict(X_test)
AB.score(X_test,y_test)

In [0]:
f1_score(y_test, y_pred, average= 'weighted')

In [0]:
from sklearn.ensemble import ExtraTreesClassifier

ET= ExtraTreesClassifier(n_estimators = 750, max_features = 'sqrt', max_depth = 35,  criterion = 'entropy', random_state = 20111974)
ET.fit(X_train,y_train)
y_pred = ET.predict(X_test)
ET.score(X_test,y_test)

In [0]:
f1_score(y_test, y_pred, average= 'weighted')

### Bagging

In [0]:
cart = DecisionTreeClassifier()
num_trees = 100
KFold= StratifiedKFold(n_splits= 10, shuffle=True)

model = BaggingClassifier(base_estimator=cart, n_estimators=num_trees)
results = model_selection.cross_val_score(model, X_train, y_train, cv=kfold)
print(results.mean())

# Evaluation
> As I'll submit a binary output, I need to use a F1-Score, as suggested in the challenge. Firstly, let's understand what's F1-Score metric:

* **Precision**: When the model predicts positive, how often is it correct? A low precision can also indicate a large number of False Positives.

    $Precision= \frac{TruePositives}{TruePositive + FalsePositives}$

* **Recall**: Recall is the number of True Positives divided by the number of True Positives and the number of False Negatives. Put another way it is the number of positive predictions divided by the number of positive class values in the test data. It is also called Sensitivity or the True Positive Rate. A low recall indicates many False Negatives.

    $Recall= \frac{TruePositives}{TruePositives + FalseNegatives}$

* **F1 Score**: F1 score conveys the balance between the precision and the recall.

    $F1= 2*\frac{Precision*Recall}{Precision+Recall}$

Source: [Classification Accuracy is Not Enough: More Performance Measures You Can Use](https://machinelearningmastery.com/classification-accuracy-is-not-enough-more-performance-measures-you-can-use/)

## Interpretation
> A good F1 score means that you have low false positives and low false negatives, so you’re correctly identifying real threats and you are not disturbed by false alarms. 
>> An F1 score is considered perfect when it’s 1, while the model is a total failure when it’s 0.

In [0]:
from sklearn.metrics import f1_score
y_pred = vc.predict(X_test)
f1_score(y_test, y_pred, average='weighted')

## Bagging

In [0]:
cart = DecisionTreeClassifier()
num_trees = 100
model = BaggingClassifier(base_estimator=cart, n_estimators=num_trees, random_state=20111974)
results = model_selection.cross_val_score(model, X_train, y_train, cv=kfold)
print(results.mean())